In [1]:
from IPython.display import display

from sympy import *

from monom import *

In [2]:
t, x, y, z, tau, h, Re, p0 = symbols(r't, x, y, z, tau, h, Re, p_0', real=True)
u, v, w, p = (f(t, x, y, z) for f in symbols('u, v, w, p', cls=Function))

In [3]:
pda_f, pda_v = (u, v, w, p), (t, x, y, z)
pda_n, pda_clp = 5, 3
pda_p = Rational(0, 1), Rational(0, 1), Rational(0, 1), Rational(0, 1)

In [4]:
Monom.variables = 1 + len(pda_v)
Monom.cmp = Monom.TOPdeglex
Monom.zero = Monom(0 for v in range(Monom.variables))
pda_fun = dict(zip(pda_f,\
             (Monom(0 if v else i for v in range(Monom.variables))\
              for i in range(1, Monom.variables))))
pda_var = dict(zip(pda_v,\
             (Monom(0 if v != i else 1 for v in range(Monom.variables))\
              for i in range(1, Monom.variables))))

In [5]:
def T(f, i1, j1, k1, l1):
    return sum(sum(sum(sum(\
        diff(f, t, i, x, j, y, k, z, l)*(tau*(i1+pda_p[0]))**i \
                                       *(h*(j1+pda_p[1]))**j \
                                       *(h*(k1+pda_p[2]))**k\
                                       *(h*(l1+pda_p[3]))**l/\
                 (factorial(i)*factorial(j)*factorial(k)*factorial(l))\
        for i in range(pda_n-j-k-l))\
        for l in range(pda_n-j-k))\
        for k in range(pda_n-j))\
        for j in range(pda_n))

In [6]:
expand((T(u, 0, 0, 0, 1)-T(u, 0, 0, 0, -1))/(2*h))

h**2*Derivative(u(t, x, y, z), (z, 3))/6 + Derivative(u(t, x, y, z), z)

In [7]:
def clip(f):
    f = f.expand()
    return [f.coeff(tau, 0).coeff(h, 0),\
            f.coeff(h, 0).coeff(tau, 1),\
            f.coeff(tau, 0).coeff(h, 2)]

def df2m(a):
    assert a.func == Derivative
    m = pda_fun[a.args[0]]
    for xi in a.args[1:]:
        if isinstance(xi, Symbol):
            m = m*pda_var[xi]
        else:
            m = m*pda_var[xi[0]]**xi[1]
    return m

def m2df(m):
    r = pda_f[m[0]-1]
    for i in range(1, len(m)):
        r = r.diff(pda_v[i-1], m[i])
    return r

def findDiv(a, d):
    r = None
    def find(a, r):
        if a.args:
            if a.func == Derivative and a.args[0] in pda_fun:
                m = df2m(a)
                if m.divisible(d) and (not r or m.cmp(r) > 0):
                    r = m
            else:
                for s in a.args:
                    r = find(s, r)
        return r
    return find(a, r)

def reduction(f1, f2, m, c, shift):
    assert shift < pda_clp
    r = [f1[i] for i in range(shift)]
    if not m:
        for i in range(shift, pda_clp):
            r.append(expand(f1[i] - f2[i-shift]*c))
    else:
        for i in range(shift, pda_clp):
            r.append(expand(f1[i] - f2[i-shift].diff(*m)*c))
    return r

def NF(f, df, G, head=False, trace=False):
    assert len(df) == len(G)
#     print(df2m(df[0]))
    ms = [df2m(d) for d in df]
    for i in range(0 if head else 1, pda_clp):
        t = 0
        if f[i]:
            while True:
                r = None
                for l in range(len(ms)):
                    r = findDiv(f[i], ms[l])
                    if r:
                        break
                if not r: 
                    break
                c, deg, m = 0, 7, m2df(r)
                while c == 0:
                    c = f[i].coeff(m, deg)
                    deg -= 1
                    assert deg >= 0
#                 print(c, m, deg+1)
                if deg:
                    c *= m**deg
                m = r/ms[l]
                d = []
                for k in range(len(pda_v)):
                    if m[k+1] > 0:
                        d.append(pda_v[k])
                        if m[k+1] > 1:
                            d.append(m[k+1])
                if trace:
                    print(">"*12)
                    eq = Symbol("eq%d" % (l+1), real=True)
                    if d:
                        display(Derivative(*tuple([eq] + d))*c*h**i)
                    else:
                        display(eq*c*h**i)
                f = reduction(f, G[l], tuple(d), c/G[l][0].coeff(df[l]), i)
                if trace:
                    print("res =")
                    display(f)
                    print("<"*12)
                t += 1
#                 if t > 6: break
    return f

def compact(f):
    def cmpct(a):
        if not a.args:
            return a
        else:
            if a in pda_fun:
                return Symbol("%s" % a.func, real=True)
            elif a.func == Derivative and a.args[0] in pda_fun:
                m = []
                for xi in a.args[1:]:
                    if isinstance(xi, Symbol):
                        m.append(str(xi))
                    else:
                        m.append(str(xi[0])*xi[1])
                return Symbol("%s_{%s}" % (a.args[0].func, "".join(m)), real=True)
            else:
                return a.func(*tuple(cmpct(s) for s in a.args))
    return cmpct(f)

def prn(a):
    display(compact(a[0]))
    print("tau =>")
    display(compact(a[1]))
    print("h^2 =>")
    display(compact(a[2]))

In [8]:
def Dt(a):
    return (T(a, 1, 0, 0, 0) - T(a, 0, 0, 0, 0))/(tau)
def Dx(a):
    return (T(a, 0, 1, 0, 0) - T(a, 0, -1, 0, 0))/(2*h)
def Dy(a):
    return (T(a, 0, 0, 1, 0) - T(a, 0, 0, -1, 0))/(2*h)
def Dz(a):
    return (T(a, 0, 0, 0, 1) - T(a, 0, 0, 0, -1))/(2*h)

def DD(a):
    return (T(a, 0, 1, 0, 0) + T(a, 0, 0, 1, 0) + T(a, 0, 0, 0, 1) +\
                -6*T(a, 0, 0, 0, 0) +\
            T(a, 0, -1, 0, 0) + T(a, 0, 0, -1, 0) + T(a, 0, 0, 0, -1))/h**2

def DD2(a):
    return (T(a, 0, 2, 0, 0) + T(a, 0, 0, 2, 0) + T(a, 0, 0, 0, 2)\
                -6*T(a, 0, 0, 0, 0) +\
           T(a, 0, -2, 0, 0) + T(a, 0, 0, -2, 0) + T(a, 0, 0, 0, -2))/(4*h**2)

def Dxx(a):
    return (T(a, 0, 2, 0, 0) - 2*T(a, 0, 0, 0, 0) + T(a, 0, -2, 0, 0))/(4*h**2)
def Dxy(a):
    return (T(a, 0, 1, 1, 0) - T(a, 0, 1, -1, 0) -\
            T(a, 0, -1, 1, 0) + T(a, 0, -1, -1, 0))/(4*h**2)
def Dxz(a):
    return (T(a, 0, 1, 0, 1) - T(a, 0, 1, 0, -1) -\
            T(a, 0, -1, 0, 1) + T(a, 0, -1, 0, -1))/(4*h**2)
def Dyy(a):
    return (T(a, 0, 0, 2, 0) - 2*T(a, 0, 0, 0, 0) + T(a, 0, 0, -2, 0))/(4*h**2)
def Dyz(a):
    return (T(a, 0, 0, 1, 1) - T(a, 0, 0, 1, -1) -\
            T(a, 0, 0, -1, 1) + T(a, 0, 0, -1, -1))/(4*h**2)

def DDx(a):
    return ((T(a, 0, 2, 0, 0) + T(a, 0, 1, 1, 0) + T(a, 0, 1, 0, 1) +\
                -6*T(a, 0, 1, 0, 0) +\
             T(a, 0, 0, 0, 0) + T(a, 0, 1, -1, 0) + T(a, 0, 1, 0, -1)) -\
            (T(a, 0, 0, 0, 0) + T(a, 0, -1, 1, 0) + T(a, 0, -1, 0, 1) +\
                -6*T(a, 0, -1, 0, 0) +\
             T(a, 0, -2, 0, 0) + T(a, 0, -1, -1, 0) + T(a, 0, -1, 0, -1)))/(2*h**3)
def DDy(a):
    return ((T(a, 0, 1, 1, 0) + T(a, 0, 0, 2, 0) + T(a, 0, 0, 1, 1) +\
                -6*T(a, 0, 0, 1, 0) +\
             T(a, 0, -1, 1, 0) + T(a, 0, 0, 0, 0) + T(a, 0, 0, 1, -1)) -\
            (T(a, 0, 1, -1, 0) + T(a, 0, 0, 0, 0) + T(a, 0, 0, -1, 1) +\
                -6*T(a, 0, 0, -1, 0) +\
             T(a, 0, -1, -1, 0) + T(a, 0, 0, -2, 0) + T(a, 0, 0, -1, -1)))/(2*h**3)
def DDz(a):
    return ((T(a, 0, 1, 0, 1) + T(a, 0, 0, 1, 1) + T(a, 0, 0, 0, 2) +\
                -6*T(a, 0, 0, 0, 1) +\
             T(a, 0, -1, 0, 1) + T(a, 0, 0, -1, 1) + T(a, 0, 0, 0, 0)) -\
            (T(a, 0, 1, 0, -1) + T(a, 0, 0, 1, -1) + T(a, 0, 0, 0, 0) +\
                -6*T(a, 0, 0, 0, -1) +\
             T(a, 0, -1, 0, -1) + T(a, 0, 0, -1, -1) + T(a, 0, 0, 0, -2)))/(2*h**3)

# FDA1

Amodio Pierluigi, Blinkov Yuri, Gerdt Vladimir, La Scala Roberto Algebraic construction and numerical behavior of a new s-consistent difference scheme for the 2D Navier–Stokes equations. Applied Mathematics and Computation, 2017, vol. 314, pp. 408-421. DOI: 10.1016/j.amc.2017.06.037 Available at: http://www.sciencedirect.com/science/article/pii/S0096300317304502

page 415

\begin{equation}
\left\lbrace 
\begin{array}{rl}
F^1:& \operatorname{D_1}(u) + \operatorname{D_2}(v) + \operatorname{D_3}(w) =0 ,\\[4pt]
F^2:& \operatorname{D_t}(u) + \operatorname{D_1}(u^2)+
\operatorname{D_2}(vu) +  \operatorname{D_3}(wu) +
\operatorname{D_1}(p) - \frac{1}{\mathrm{Re}} \tilde{\Delta} (u)
=0 ,\\[4pt]
F^3:& \operatorname{D_t}(v) + \operatorname{D_1}(uv)+
\operatorname{D_2}(v^2)  +  \operatorname{D_3}(wv) +
\operatorname{D_2}(p) - \frac{1}{\mathrm{Re}} \tilde{\Delta} (v)
=0 ,\\[4pt]
F^4:& \operatorname{D_t}(w) + \operatorname{D_1}(uw)+
\operatorname{D_2}(vw)  +  \operatorname{D_3}(w^2) +
\operatorname{D_3}(p) - \frac{1}{\mathrm{Re}} \tilde{\Delta} (w)
=0 ,\\[4pt]
F^5:& \left(\operatorname{D_1}^2 + \operatorname{D_2}^2 + \operatorname{D_3}^2\right)(p) 
+ \operatorname{D_1}^2(u^2) + \operatorname{D_2}^2(v^2) + \operatorname{D_3}^2(w^2) + {} \\
&{} + 2\operatorname{D_1}\operatorname{D_2}(uv) + 2\operatorname{D_1}\operatorname{D_3}(uw)
+ 2\operatorname{D_2}\operatorname{D_3}(vw)
- \frac{1}{\mathrm{Re}}\tilde{\Delta}(F^1)
=0 .
\end{array}
\right. 
\end{equation}

In [9]:
F1 = clip(Dx(u) + Dy(v) + Dz(w))
F2 = clip(Dt(u) + Dx(u**2) + Dy(v*u) + Dz(w*u) +  Dx(p) - DD(u)/Re)
F3 = clip(Dt(v) + Dx(u*v) +  Dy(v**2) + Dz(w*v) + Dy(p) - DD(v)/Re)
F4 = clip(Dt(w) + Dx(u*w) +  Dy(v*w) + Dz(w**2) + Dy(p) - DD(w)/Re)
F5 = clip(DD2(p) + Dxx(u**2) + Dyy(v**2) + Dyy(w**2) + \
          2*Dxy(u*v) + 2*Dxz(u*w) + 2*Dyz(v*w) - (DDx(u) + DDy(v) + DDz(w))/Re)

In [10]:
f5 = NF(F5, [u.diff(x), u.diff(y, 2), v.diff(x, 2), w.diff(x, 2)], [F1, F2, F3, F4], head=True)
f5 = NF(f5, [u.diff(x), u.diff(y, 2), v.diff(x, 2), w.diff(x, 2), p.diff(x, 2)], [F1, F2, F3, F4, f5])
prn(f5)

p_{xx} + p_{yy} - p_{yz} + p_{zz} + u*w_{xz} + 2*u_{y}*v_{x} + u_{z}*w_{x} + v*w_{yz} + 2*v_{y}**2 + 2*v_{y} + v_{z}*w_{y} + 2*w*w_{yy} - 2*w*w_{zz} - w_{tz} + 2*w_{y}**2 - 2*w_{z}**2 - w_{z} + 2

tau =>


-w_{ttz}/2

h^2 =>


4*Re**2*p_{x}*v*v_{x}/3 - 2*Re**2*p_{x}*w*w_{x}/3 - Re**2*p_{yy}*u**2/6 - Re**2*p_{yz}*u**2/12 - 2*Re**2*p_{y}*u*u_{y}/3 - Re**2*p_{y}*u*u_{z}/6 - Re**2*u**3*v_{xy}/6 - Re**2*u**3*w_{xz}/12 - 5*Re**2*u**2*u_{y}*v_{x}/6 - Re**2*u**2*u_{z}*w_{x}/4 - Re**2*u**2*v*v_{yy}/6 - Re**2*u**2*v*w_{yz}/4 - Re**2*u**2*v_{ty}/6 - Re**2*u**2*v_{yz}*w/6 - Re**2*u**2*v_{y}**2/6 - Re**2*u**2*v_{y}*w_{z}/6 + Re**2*u**2*v_{y}/6 - Re**2*u**2*v_{z}*w_{y}/4 - Re**2*u**2*w*w_{zz}/6 - Re**2*u**2*w_{tz}/12 - Re**2*u**2*w_{z}**2/6 + Re**2*u**2*w_{z}/12 - 2*Re**2*u*u_{y}*v*v_{y}/3 - 2*Re**2*u*u_{y}*v*w_{z}/3 + 2*Re**2*u*u_{y}*v/3 - 2*Re**2*u*u_{y}*v_{t}/3 - 2*Re**2*u*u_{y}*v_{z}*w/3 - Re**2*u*u_{z}*v*w_{y}/6 - Re**2*u*u_{z}*w*w_{z}/3 + Re**2*u*u_{z}*w/6 - Re**2*u*u_{z}*w_{t}/6 - 4*Re**2*u*v*v_{x}*v_{y}/3 + 4*Re**2*u*v*v_{x}*w_{z}/3 - 8*Re**2*u*v*v_{x}/3 + 2*Re**2*u*v_{y}*w*w_{x}/3 - 2*Re**2*u*w*w_{x}*w_{z}/3 + 4*Re**2*u*w*w_{x}/3 + 4*Re**2*u_{t}*v*v_{x}/3 - 2*Re**2*u_{t}*w*w_{x}/3 + 4*Re**2*u_{y}*v**2*v_{x}/3 - 2

In [11]:
f1 = NF(F1, [u.diff(y, 2), v.diff(x, 2), w.diff(x, 2), p.diff(x, 2)], [F2, F3, F4, f5], head=True)
f1 = NF(f1, [u.diff(x), u.diff(y, 2), v.diff(x, 2), w.diff(x, 2), p.diff(x, 2)], [f1, F2, F3, F4, f5])
prn(f1)

u_{x} + v_{y} + 1

tau =>


0

h^2 =>


-Re*p_{yy}/6 - Re*u*v_{xy}/6 - Re*u_{y}*v_{x}/6 - Re*v*v_{yy}/6 - Re*v*w_{yz}/6 - Re*v_{ty}/6 - Re*v_{yz}*w/6 - Re*v_{y}**2/6 - Re*v_{y}*w_{z}/6 + Re*v_{y}/6 - Re*v_{z}*w_{y}/6 + v_{yyy}/3 + v_{yzz}/6

In [12]:
f2 = NF(F2, [u.diff(x), v.diff(x, 2), w.diff(x, 2), p.diff(x, 2)], [f1, F3, F4, f5], head=True)
f2 = NF(f2, [u.diff(x), u.diff(y, 2), v.diff(x, 2), w.diff(x, 2), p.diff(x, 2)], [f1, f2, F3, F4, f5])
prn(f2)

p_{x} - u*v_{y} + u*w_{z} - 2*u + u_{t} + u_{y}*v + u_{z}*w - u_{yy}/Re - u_{zz}/Re + v_{xy}/Re

tau =>


u_{tt}/2

h^2 =>


Re**2*p_{x}*v**2/12 - Re**2*u*v**2*v_{y}/12 + Re**2*u*v**2*w_{z}/12 - Re**2*u*v**2/6 + Re**2*u_{t}*v**2/12 + Re**2*u_{y}*v**3/12 + Re**2*u_{z}*v**2*w/12 - Re*p_{tx}/12 + Re*p_{xy}*v/12 - Re*p_{xz}*w/12 + 5*Re*p_{x}*v_{y}/12 - Re*p_{x}*w_{z}/12 + Re*p_{x}/6 - Re*p_{yy}*u/12 - Re*p_{yz}*u/6 - 5*Re*p_{y}*u_{y}/12 - Re*p_{y}*u_{z}/6 - Re*u**2*v_{xy}/12 - Re*u**2*w_{xz}/6 - Re*u*u_{y}*v_{x}/2 - Re*u*u_{z}*w_{x}/3 - Re*u*v*v_{yy}/6 - Re*u*v*w_{yz}/6 - Re*u*v_{y}**2/2 + 5*Re*u*v_{y}*w_{z}/12 - 11*Re*u*v_{y}/12 - Re*u*v_{z}*w_{y}/4 - 5*Re*u*w*w_{zz}/12 - Re*u*w_{tz}/4 - 5*Re*u*w_{z}**2/12 + Re*u*w_{z}/2 - Re*u/3 - Re*u_{tt}/12 - Re*u_{tz}*w/6 + Re*u_{t}*v_{y}/2 - Re*u_{t}*w_{z}/6 + Re*u_{t}/3 - 5*Re*u_{y}*v*w_{z}/12 + 5*Re*u_{y}*v/12 - Re*u_{y}*v_{t}/2 - Re*u_{y}*v_{z}*w/2 - Re*u_{zz}*v**2/12 - Re*u_{zz}*w**2/12 - Re*u_{z}*v*w_{y}/12 + Re*u_{z}*v_{y}*w/2 - 7*Re*u_{z}*w*w_{z}/12 + Re*u_{z}*w/2 - Re*u_{z}*w_{t}/4 + Re*v**2*v_{xy}/12 - p_{xyy}/3 + p_{xyz}/6 - p_{xzz}/12 + u*w_{yyz}/12 + 5*u*w_{zz

In [13]:
f3 = NF(F3, [u.diff(x), u.diff(y, 2), w.diff(x, 2), p.diff(x, 2)], [f1, f2, F4, f5], head=True)
f3 = NF(f3, [u.diff(x), u.diff(y, 2), v.diff(x, 2), w.diff(x, 2), p.diff(x, 2)], [f1, f2, f3, F4, f5])
prn(f3)

p_{y} + u*v_{x} + v*v_{y} + v*w_{z} - v + v_{t} + v_{z}*w - v_{xx}/Re - v_{yy}/Re - v_{zz}/Re

tau =>


v_{tt}/2

h^2 =>


Re**2*p_{y}*u**2/12 + Re**2*u**3*v_{x}/12 + Re**2*u**2*v*v_{y}/12 + Re**2*u**2*v*w_{z}/12 - Re**2*u**2*v/12 + Re**2*u**2*v_{t}/12 + Re**2*u**2*v_{z}*w/12 - Re*p_{ty}/12 + Re*p_{xy}*u/12 + Re*p_{x}*v_{x}/4 - Re*p_{yy}*v/12 - Re*p_{yz}*v/12 - Re*p_{yz}*w/12 - 5*Re*p_{y}*v_{y}/12 - Re*p_{y}*v_{z}/12 - Re*p_{y}*w_{z}/12 - Re*p_{y}/4 - Re*u**2*v_{yy}/12 - Re*u**2*v_{zz}/12 - 2*Re*u*v_{x}*v_{y}/3 + Re*u*v_{x}*w_{z}/4 - 11*Re*u*v_{x}/12 + Re*u_{t}*v_{x}/6 + Re*u_{y}*v*v_{x}/6 - Re*u_{z}*v*w_{x}/12 + Re*u_{z}*v_{x}*w/6 - Re*v**2*v_{yy}/12 - Re*v**2*w_{yz}/6 - Re*v*v_{ty}/6 - Re*v*v_{yz}*w/6 - Re*v*v_{y}**2/2 - 7*Re*v*v_{y}*w_{z}/12 + Re*v*v_{y}/4 - Re*v*v_{z}*w_{y}/4 - Re*v*w*w_{zz}/4 - Re*v*w_{tz}/6 - Re*v*w_{z}**2/4 - Re*v*w_{z}/12 + Re*v/4 - Re*v_{tt}/12 - Re*v_{tz}*w/6 - Re*v_{t}*v_{y}/2 - Re*v_{t}*w_{z}/6 - Re*v_{t}/6 - Re*v_{y}*v_{z}*w/2 - Re*v_{zz}*w**2/12 - 5*Re*v_{z}*w*w_{z}/12 - Re*v_{z}*w/12 - Re*v_{z}*w_{t}/6 + p_{yyy}/3 - p_{yyz}/12 + p_{yzz}/6 + u*w_{xyz}/12 + u_{yz}*w_{x}/12 + u

In [14]:
f4 = NF(F4, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [f1, f2, f3, f5], head=True)
f4 = NF(f4, [u.diff(x), u.diff(y, 2), v.diff(x, 2), w.diff(x, 2), p.diff(x, 2)], [f1, f2, f3, f4, f5])
prn(f4)

p_{y} + u*w_{x} + v*w_{y} + 2*w*w_{z} - w + w_{t} - w_{xx}/Re - w_{yy}/Re - w_{zz}/Re

tau =>


w_{tt}/2

h^2 =>


Re**2*p_{y}*u**2/12 + Re**2*u**3*w_{x}/12 + Re**2*u**2*v*w_{y}/12 + Re**2*u**2*w*w_{z}/6 - Re**2*u**2*w/12 + Re**2*u**2*w_{t}/12 - Re*p_{ty}/12 + Re*p_{xy}*u/12 + Re*p_{x}*v_{x}/6 + Re*p_{x}*w_{x}/12 - Re*p_{yy}*v/12 - Re*p_{yz}*w/6 - Re*p_{y}*v_{y}/3 - Re*p_{y}*w_{y}/12 - Re*p_{y}*w_{z}/6 - Re*p_{y}/4 - Re*u**2*w_{yy}/12 - Re*u**2*w_{zz}/12 - Re*u*v_{x}*v_{y}/6 + Re*u*v_{x}*w_{z}/6 - Re*u*v_{x}/3 - Re*u*v_{y}*w_{x}/2 + Re*u*w_{x}*w_{z}/12 - 7*Re*u*w_{x}/12 + Re*u_{t}*v_{x}/6 + Re*u_{y}*v*v_{x}/6 + Re*u_{z}*v_{x}*w/6 - Re*u_{z}*w*w_{x}/12 - Re*v**2*w_{yy}/12 - Re*v*v_{y}*w_{y}/2 - Re*v*w*w_{yz}/3 - Re*v*w_{ty}/6 - 5*Re*v*w_{y}*w_{z}/12 - Re*v*w_{y}/12 - Re*v_{t}*w_{y}/6 - 2*Re*v_{y}*w*w_{z}/3 + Re*v_{y}*w/3 - Re*v_{y}*w_{t}/3 - Re*v_{z}*w*w_{y}/4 - Re*w**2*w_{zz}/3 - Re*w*w_{tz}/3 - 2*Re*w*w_{z}**2/3 - Re*w*w_{z}/6 + Re*w/4 - Re*w_{tt}/12 - Re*w_{t}*w_{z}/3 - Re*w_{t}/6 + p_{yyy}/3 - p_{yyz}/12 + p_{yzz}/6 + u*w_{xyz}/12 + u_{yz}*w_{x}/12 + u_{y}*v_{xy}/6 + u_{y}*w_{xy}/6 + u_{y}*w_{xz

In [15]:
spoly = [a.diff(x) + b.diff(y) for a, b in zip(f1, f2)]
spoly = NF(spoly, [u.diff(x), u.diff(y, 2), v.diff(x, 2), w.diff(x, 2),\
                   p.diff(x, 2)], [f1, f2, f3, f4, f5], head=True)
prn(spoly)

0

tau =>


0

h^2 =>


0

# FDA2

Hans Johnston and Jian-Guo Liu, Finite Difference Schemes for Incompressible Flow Based on Local Pressure Boundary Conditions. Journal of Computational Physics 180, 120–154 (2002) doi:10.1006/jcph.2002.7079

page 126

\begin{equation}
\left\lbrace 
\begin{array}{rl}
F^1:& \operatorname{D_1}(u) + \operatorname{D_2}(v) + \operatorname{D_3}(w)=0 ,\\[4pt]
F^2:& \operatorname{D_t}(u) + u\operatorname{D_1}(u)+
v\operatorname{D_2}(u) + w\operatorname{D_3}(u) +
\operatorname{D_1}(p) - \frac{1}{\mathrm{Re}} \tilde{\Delta} (u)
=0 ,\\[4pt]
F^3:& \operatorname{D_t}(v) + u\operatorname{D_1}(v)+
v\operatorname{D_2}(v) + w\operatorname{D_3}(w) +
\operatorname{D_2}(p) - \frac{1}{\mathrm{Re}} \tilde{\Delta} (v)
=0 ,\\[4pt]
F^4:& \operatorname{D_t}(w) + u\operatorname{D_1}(w)+
v\operatorname{D_2}(w) + w\operatorname{D_3}(w) +
\operatorname{D_3}(p) - \frac{1}{\mathrm{Re}} \tilde{\Delta} (w)
=0 ,\\[4pt]
F^5:& \tilde{\Delta}(p) 
- 2\operatorname{D_1}(u)\operatorname{D_2}(v)
+ 2\operatorname{D_1}(v)\operatorname{D_2}(u)
=0 .
\end{array}
\right. 
\end{equation}

In [27]:
F1 = clip(Dx(u) + Dy(v))
F2 = clip(Dt(u) + u*Dx(u) + v*Dy(u) + Dx(p) - DD(u)/Re)
F3 = clip(Dt(v) + u*Dx(v) + v*Dy(v) + Dy(p) - DD(v)/Re)
F4 = clip(DD2(p) - 2*Dx(u)*Dy(v) + 2*Dx(v)*Dy(u))

In [32]:
f4 = NF(F4, [u.diff(x), u.diff(y, 2), v.diff(x, 2)], [F1, F2, F3], head=True)
f4 = NF(f4, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [F1, F2, F3, f4])
prn(f4)

p_{xx} + p_{yy} + 2*u_{y}*v_{x} + 2*v_{y}**2

tau =>


0

h^2 =>


Re**2*p_{x}*v*v_{x} - Re**2*p_{y}*u*u_{y}/3 - Re**2*u**2*u_{y}*v_{x}/3 - Re**2*u*u_{y}*v*v_{y}/3 - Re**2*u*u_{y}*v_{t}/3 - Re**2*u*v*v_{x}*v_{y} + Re**2*u_{t}*v*v_{x} + Re**2*u_{y}*v**2*v_{x} - Re*p_{xy}*u_{y}/3 + Re*p_{xy}*v_{x} + 4*Re*p_{x}*v_{xy}/3 - 4*Re*p_{yy}*v_{y}/3 + 4*Re*p_{y}*v_{yy}/3 + Re*u*u_{y}*v_{yy}/3 - 8*Re*u*v_{xy}*v_{y}/3 + Re*u*v_{x}*v_{yy}/3 + Re*u_{ty}*v_{x} + 4*Re*u_{t}*v_{xy}/3 + Re*u_{y}*v*v_{xy} - Re*u_{y}*v_{tx}/3 - 4*Re*u_{y}*v_{x}*v_{y}/3 + Re*v*v_{xy}*v_{x} - 4*Re*v_{ty}*v_{y}/3 + 4*Re*v_{t}*v_{yy}/3 - 4*Re*v_{y}**3/3 + 2*p_{yyyy}/3 + u_{y}*v_{xyy} + 5*v_{xyy}*v_{x}/3 + 10*v_{yyy}*v_{y}/3

In [34]:
f1 = NF(F1, [u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [F2, F3, f4], head=True)
f1 = NF(f1, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [f1, F2, F3, f4])
prn(f1)

u_{x} + v_{y}

tau =>


0

h^2 =>


-Re*p_{yy}/6 - Re*u*v_{xy}/6 - Re*u_{y}*v_{x}/6 - Re*v*v_{yy}/6 - Re*v_{ty}/6 - Re*v_{y}**2/6 + v_{yyy}/3

In [36]:
f2 = NF(F2, [u.diff(x), v.diff(x, 2), p.diff(x, 2)], [f1, F3, f4], head=True)
f2 = NF(f2, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [f1, f2, F3, f4])
prn(f2)

p_{x} - u*v_{y} + u_{t} + u_{y}*v - u_{yy}/Re + v_{xy}/Re

tau =>


u_{tt}/2

h^2 =>


Re**2*p_{x}*v**2/12 - Re**2*u*v**2*v_{y}/12 + Re**2*u_{t}*v**2/12 + Re**2*u_{y}*v**3/12 - Re*p_{tx}/12 + Re*p_{xy}*v/12 - Re*p_{x}*v_{y}/12 - Re*p_{yy}*u/12 - 5*Re*p_{y}*u_{y}/12 - Re*u**2*v_{xy}/12 - Re*u*u_{y}*v_{x}/2 - Re*u*v*v_{yy}/6 - Re*u_{tt}/12 - Re*u_{y}*v*v_{y}/2 - Re*u_{y}*v_{t}/2 + Re*v**2*v_{xy}/12 - p_{xyy}/3 + u_{y}*v_{yy}/2 - v_{txy}/6 - 5*v_{xy}*v_{y}/6 + v_{x}*v_{yy}/3 + v_{xyyy}/(6*Re)

In [38]:
f3 = NF(F3, [u.diff(x), u.diff(y, 2), p.diff(x, 2)], [f1, f2, f4], head=True)
f3 = NF(f3, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [f1, f2, f3, f4])
prn(f3)

p_{y} + u*v_{x} + v*v_{y} + v_{t} - v_{xx}/Re - v_{yy}/Re

tau =>


v_{tt}/2

h^2 =>


Re**2*p_{y}*u**2/12 + Re**2*u**3*v_{x}/12 + Re**2*u**2*v*v_{y}/12 + Re**2*u**2*v_{t}/12 - Re*p_{ty}/12 + Re*p_{xy}*u/12 + Re*p_{x}*v_{x}/4 - Re*p_{yy}*v/12 + Re*p_{y}*v_{y}/12 - Re*u**2*v_{yy}/12 - Re*u*v_{x}*v_{y}/6 + Re*u_{t}*v_{x}/6 + Re*u_{y}*v*v_{x}/6 - Re*v**2*v_{yy}/12 - Re*v*v_{ty}/6 - Re*v_{tt}/12 + p_{yyy}/3 + u_{y}*v_{xy}/3 + v*v_{yyy}/3 + v_{tyy}/6 + v_{xy}*v_{x}/6 + v_{yy}*v_{y}/2 - v_{yyyy}/(6*Re)

In [40]:
spoly = [a.diff(x) + b.diff(y) for a, b in zip(f2, f3)]
spoly = NF(spoly, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [f1, f2, f3, f4], head=True)
prn(spoly)

0

tau =>


0

h^2 =>


Re*p_{x}*v_{xy}/2 + Re*p_{yy}*v_{y}/2 - Re*p_{y}*v_{yy}/2 - Re*u*v_{x}*v_{yy}/2 + Re*u_{t}*v_{xy}/2 + Re*u_{y}*v*v_{xy}/2 + Re*u_{y}*v_{x}*v_{y}/2 + Re*v_{ty}*v_{y}/2 - Re*v_{t}*v_{yy}/2 + Re*v_{y}**3/2 + u_{y}*v_{xyy}/2 - v_{xyy}*v_{x}/2 + v_{xy}**2 + v_{yy}**2

# FDA3

Amodio Pierluigi, Blinkov Yuri, Gerdt Vladimir, La Scala Roberto Algebraic construction and numerical behavior of a new s-consistent difference scheme for the 2D Navier–Stokes equations. Applied Mathematics and Computation, 2017, vol. 314, pp. 408-421. DOI: 10.1016/j.amc.2017.06.037 

Available at: http://www.sciencedirect.com/science/article/pii/S0096300317304502

page 417

\begin{equation}
\left\lbrace 
\begin{array}{rl}
F^1:& \operatorname{D_1}(u) + \operatorname{D_2}(v) + \operatorname{D_3}(w) =0 ,\\[4pt]
F^2:& \operatorname{D_t}(u) + \operatorname{D_1}(u^2)+
\operatorname{D_2}(vu) +  \operatorname{D_3}(wu) +
\operatorname{D_1}(p) - \frac{1}{\mathrm{Re}} \tilde{\Delta} (u)
=0 ,\\[4pt]
F^3:& \operatorname{D_t}(v) + \operatorname{D_1}(uv)+
\operatorname{D_2}(v^2)  +  \operatorname{D_3}(wv) +
\operatorname{D_2}(p) - \frac{1}{\mathrm{Re}} \tilde{\Delta} (v)
=0 ,\\[4pt]
F^4:& \operatorname{D_t}(w) + \operatorname{D_1}(uw)+
\operatorname{D_2}(vw)  +  \operatorname{D_3}(w^2) +
\operatorname{D_3}(p) - \frac{1}{\mathrm{Re}} \tilde{\Delta} (w)
=0 ,\\[4pt]
F^5:& \left(\operatorname{D_1}^2 + \operatorname{D_2}^2 + \operatorname{D_3}^2\right)(p) 
+ \operatorname{D_1}^2(u^2) + \operatorname{D_2}^2(v^2) + \operatorname{D_3}^2(w^2) + {} \\
&{} + 2\operatorname{D_1}\operatorname{D_2}(uv) + 2\operatorname{D_1}\operatorname{D_3}(uw)
+ 2\operatorname{D_2}\operatorname{D_3}(vw)
=0 .
\end{array}
\right. 
\end{equation}

In [20]:
F1 = clip(Dx(u) + Dy(v) + Dz(w))
F2 = clip(Dt(u) + Dx(u**2) + Dy(v*u) +  Dz(w*u) +  Dx(p) - DD(u)/Re)
F3 = clip(Dt(v) + Dx(u*v) +  Dy(v**2) + Dz(w*v) +  Dy(p) - DD(v)/Re)
F4 = clip(Dt(w) + Dx(u*w) +  Dy(v*w) +  Dz(w**2) + Dy(p) - DD(w)/Re)
F5 = clip(DD2(p) + Dxx(u**2) + Dyy(v**2) + Dyy(w**2) + \
          2*Dxy(u*v) + 2*Dxz(u*w) + 2*Dyz(v*w))

In [21]:
f5 = NF(F5, [u.diff(x), u.diff(y, 2), v.diff(x, 2), w.diff(x, 2)], [F1, F2, F3, F4], head=True)
f5 = NF(f5, [u.diff(x), u.diff(y, 2), v.diff(x, 2), w.diff(x, 2), p.diff(x, 2)], [F1, F2, F3, F4, f5])
prn(f5)

p_{xx} + p_{yy} + p_{zz} + 2*u_{y}*v_{x} + 2*u_{z}*w_{x} + 2*v_{y}**2 + 2*v_{y}*w_{z} + 2*v_{z}*w_{y} + 2*w*w_{yy} - 2*w*w_{zz} + 2*w_{y}**2

tau =>


0

h^2 =>


Re**2*p_{x}*v*v_{x} - 2*Re**2*p_{x}*w*w_{x}/3 - Re**2*p_{y}*u*u_{y}/3 - Re**2*p_{y}*u*u_{z}/3 - Re**2*u**2*u_{y}*v_{x}/3 - Re**2*u**2*u_{z}*w_{x}/3 - Re**2*u*u_{y}*v*v_{y}/3 - Re**2*u*u_{y}*v_{t}/3 - Re**2*u*u_{y}*v_{z}*w/3 - Re**2*u*u_{z}*v*w_{y}/3 - Re**2*u*u_{z}*w*w_{z}/3 - Re**2*u*u_{z}*w_{t}/3 - Re**2*u*v*v_{x}*v_{y} - Re**2*u*v*v_{x}*w_{z} + 2*Re**2*u*v_{y}*w*w_{x}/3 + 2*Re**2*u*w*w_{x}*w_{z}/3 + Re**2*u_{t}*v*v_{x} - 2*Re**2*u_{t}*w*w_{x}/3 + Re**2*u_{y}*v**2*v_{x} - 2*Re**2*u_{y}*v*w*w_{x}/3 + Re**2*u_{z}*v*v_{x}*w - 2*Re**2*u_{z}*w**2*w_{x}/3 - Re*p_{xy}*u_{y}/3 - Re*p_{xy}*u_{z}/3 + Re*p_{xy}*v_{x} + 2*Re*p_{xz}*w_{x}/3 + 7*Re*p_{x}*v_{xy}/3 - 2*Re*p_{yyy}*w/3 - Re*p_{yy}*v_{y}/3 - 2*Re*p_{yy}*v_{z}/3 - 4*Re*p_{yy}*w_{y}/3 + Re*p_{yy}*w_{z}/3 + 2*Re*p_{yzz}*w/3 + Re*p_{yz}*v_{y}/3 - 2*Re*p_{yz}*w_{y}/3 + Re*p_{yz}*w_{z} + Re*p_{y}*v_{yy}/3 + Re*p_{y}*v_{yz}/3 - 2*Re*p_{y}*w_{yy}/3 + Re*p_{y}*w_{yz}/3 + Re*p_{y}*w_{zz} + Re*u*u_{y}*v_{yy}/3 + Re*u*u_{y}*v_{zz}/3 + Re*u*u_{z}*w

In [22]:
f1 = NF(F1, [u.diff(y, 2), v.diff(x, 2), w.diff(x, 2), p.diff(x, 2)], [F2, F3, F4, f5], head=True)
f1 = NF(f1, [u.diff(x), u.diff(y, 2), v.diff(x, 2), w.diff(x, 2), p.diff(x, 2)], [f1, F2, F3, F4, f5])
prn(f1)

u_{x} + v_{y} + w_{z}

tau =>


0

h^2 =>


-Re*p_{yy}/6 - Re*p_{yz}/6 - Re*u*v_{xy}/6 - Re*u*w_{xz}/6 - Re*u_{y}*v_{x}/6 - Re*u_{z}*w_{x}/6 - Re*v*v_{yy}/6 - Re*v*w_{yz}/6 - Re*v_{ty}/6 - Re*v_{yz}*w/6 - Re*v_{y}**2/6 - Re*v_{z}*w_{y}/3 - Re*w*w_{zz}/6 - Re*w_{tz}/6 - Re*w_{z}**2/6 + v_{yyy}/3 + v_{yzz}/6 + w_{yyz}/6 + w_{zzz}/3

In [23]:
f2 = NF(F2, [u.diff(x), v.diff(x, 2), w.diff(x, 2), p.diff(x, 2)], [f1, F3, F4, f5], head=True)
f2 = NF(f2, [u.diff(x), u.diff(y, 2), v.diff(x, 2), w.diff(x, 2), p.diff(x, 2)], [f1, f2, F3, F4, f5])
prn(f2)

p_{x} - u*v_{y} - u*w_{z} + u_{t} + u_{y}*v + u_{z}*w - u_{yy}/Re - u_{zz}/Re + v_{xy}/Re + w_{xz}/Re

tau =>


u_{tt}/2

h^2 =>


Re**2*p_{x}*v**2/12 - Re**2*u*v**2*v_{y}/12 - Re**2*u*v**2*w_{z}/12 + Re**2*u_{t}*v**2/12 + Re**2*u_{y}*v**3/12 + Re**2*u_{z}*v**2*w/12 - Re*p_{tx}/12 + Re*p_{xy}*v/12 - Re*p_{xz}*w/12 + 5*Re*p_{x}*v_{y}/12 + Re*p_{x}*w_{z}/12 - Re*p_{yy}*u/12 - Re*p_{yz}*u/12 - 5*Re*p_{y}*u_{y}/12 - 5*Re*p_{y}*u_{z}/12 - Re*u**2*v_{xy}/12 - Re*u**2*w_{xz}/12 - Re*u*u_{y}*v_{x}/2 - Re*u*u_{z}*w_{x}/2 - Re*u*v*v_{yy}/6 - Re*u*v*w_{yz}/6 - Re*u*v_{y}**2/2 - Re*u*v_{y}*w_{z}/2 - Re*u*v_{z}*w_{y}/6 - Re*u*w_{z}**2/6 - Re*u_{tt}/12 - Re*u_{tz}*w/6 + Re*u_{t}*v_{y}/2 + Re*u_{t}*w_{z}/6 - Re*u_{y}*v_{t}/2 - Re*u_{y}*v_{z}*w/2 - Re*u_{zz}*v**2/12 - Re*u_{zz}*w**2/12 - Re*u_{z}*v*w_{y}/3 + Re*u_{z}*v_{y}*w/2 - Re*u_{z}*w*w_{z}/3 - Re*u_{z}*w_{t}/2 + Re*v**2*v_{xy}/12 + Re*v**2*w_{xz}/12 - p_{xyy}/3 - p_{xyz}/12 - p_{xzz}/12 + u*w_{yyz}/6 - u*w_{zzz}/6 + u_{tzz}/6 + u_{yz}*v_{z}/6 - u_{yz}*w_{y}/6 + u_{y}*v_{yy} + u_{y}*v_{zz}/2 + u_{y}*w_{yz}/6 + u_{zzz}*w/3 - u_{zz}*v_{y}/2 + u_{zz}*w_{z}/2 - u_{z}*v_{yz}/6 + 

In [24]:
f3 = NF(F3, [u.diff(x), u.diff(y, 2), w.diff(x, 2), p.diff(x, 2)], [f1, f2, F4, f5], head=True)
f3 = NF(f3, [u.diff(x), u.diff(y, 2), v.diff(x, 2), w.diff(x, 2), p.diff(x, 2)], [f1, f2, f3, F4, f5])
prn(f3)

p_{y} + u*v_{x} + v*v_{y} + v_{t} + v_{z}*w - v_{xx}/Re - v_{yy}/Re - v_{zz}/Re

tau =>


v_{tt}/2

h^2 =>


Re**2*p_{y}*u**2/12 + Re**2*u**3*v_{x}/12 + Re**2*u**2*v*v_{y}/12 + Re**2*u**2*v_{t}/12 + Re**2*u**2*v_{z}*w/12 - Re*p_{ty}/12 + Re*p_{xy}*u/12 + Re*p_{x}*v_{x}/4 - Re*p_{yy}*v/12 - Re*p_{yz}*w/12 - 5*Re*p_{y}*v_{y}/12 - Re*p_{y}*v_{z}/12 - Re*p_{y}*w_{z}/3 - Re*u**2*v_{yy}/12 - Re*u**2*v_{zz}/12 - 2*Re*u*v_{x}*v_{y}/3 - 2*Re*u*v_{x}*w_{z}/3 + Re*u_{t}*v_{x}/6 + Re*u_{y}*v*v_{x}/6 + Re*u_{z}*v_{x}*w/6 - Re*v**2*v_{yy}/12 - Re*v*v_{ty}/6 - Re*v*v_{yz}*w/6 - Re*v*v_{y}**2/2 - Re*v*v_{y}*w_{z}/3 - Re*v*v_{z}*w_{y}/6 - Re*v_{tt}/12 - Re*v_{tz}*w/6 - Re*v_{t}*v_{y}/2 - Re*v_{t}*w_{z}/3 - Re*v_{y}*v_{z}*w/2 - Re*v_{zz}*w**2/12 - Re*v_{z}*w*w_{z}/2 - Re*v_{z}*w_{t}/6 + p_{yyy}/3 + p_{yzz}/6 + u_{yz}*w_{x}/6 + u_{y}*v_{xy}/3 - u_{zz}*v_{x}/6 + u_{z}*v_{xz}/6 + u_{z}*w_{xy}/6 + v*v_{yyy}/3 + v*v_{yzz}/6 + v_{tyy}/6 + v_{tzz}/6 - v_{xy}*v_{x}/3 - v_{xz}*w_{x}/6 - v_{x}*w_{xz}/6 + v_{yyz}*w/6 + 2*v_{yy}*v_{y} + v_{yy}*w_{z}/2 + v_{yz}*v_{z}/6 + v_{yz}*w_{y}/3 + v_{y}*v_{zz}/2 + v_{y}*w_{yz}/6 + v

In [25]:
f4 = NF(F4, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [f1, f2, f3, f5], head=True)
f4 = NF(f4, [u.diff(x), u.diff(y, 2), v.diff(x, 2), w.diff(x, 2), p.diff(x, 2)], [f1, f2, f3, f4, f5])
prn(f4)

p_{y} + u*w_{x} + v*w_{y} + w*w_{z} + w_{t} - w_{xx}/Re - w_{yy}/Re - w_{zz}/Re

tau =>


w_{tt}/2

h^2 =>


Re**2*p_{y}*u**2/12 + Re**2*u**3*w_{x}/12 + Re**2*u**2*v*w_{y}/12 + Re**2*u**2*w*w_{z}/12 + Re**2*u**2*w_{t}/12 - Re*p_{ty}/12 + Re*p_{xy}*u/12 + Re*p_{x}*v_{x}/6 + Re*p_{x}*w_{x}/12 - Re*p_{yy}*v/12 - Re*p_{yz}*w/12 - Re*p_{y}*v_{y}/3 - Re*p_{y}*w_{y}/12 - 5*Re*p_{y}*w_{z}/12 - Re*u**2*w_{yy}/12 - Re*u**2*w_{zz}/12 - Re*u*v_{x}*v_{y}/6 - Re*u*v_{x}*w_{z}/6 - Re*u*v_{y}*w_{x}/2 - Re*u*w_{x}*w_{z}/2 + Re*u_{t}*v_{x}/6 + Re*u_{y}*v*v_{x}/6 + Re*u_{z}*v_{x}*w/6 - Re*v**2*w_{yy}/12 - Re*v*v_{y}*w_{y}/2 - Re*v*w*w_{yz}/6 - Re*v*w_{ty}/6 - Re*v*w_{y}*w_{z}/2 - Re*v_{t}*w_{y}/6 - Re*v_{y}*w*w_{z}/3 - Re*v_{y}*w_{t}/3 - Re*v_{z}*w*w_{y}/6 - Re*w**2*w_{zz}/12 - Re*w*w_{tz}/6 - Re*w*w_{z}**2/2 - Re*w_{tt}/12 - Re*w_{t}*w_{z}/2 + p_{yyy}/3 + p_{yzz}/6 + u_{yz}*w_{x}/6 + u_{y}*v_{xy}/6 + u_{y}*w_{xy}/6 - u_{zz}*v_{x}/6 + u_{z}*w_{xy}/6 + u_{z}*w_{xz}/6 + v*w_{yyy}/3 + v*w_{yzz}/6 + v_{xy}*v_{x}/6 - v_{xy}*w_{x}/3 - v_{x}*w_{xy}/6 + v_{x}*w_{xz}/6 + v_{yy}*v_{y}/3 + 2*v_{yy}*w_{y}/3 + v_{yy}*w_{z}/

In [26]:
spoly = [a.diff(x) + b.diff(y) for a, b in zip(f1, f2)]
spoly = NF(spoly, [u.diff(x), u.diff(y, 2), v.diff(x, 2), w.diff(x, 2),\
                   p.diff(x, 2)], [f1, f2, f3, f4, f5], head=True)
prn(spoly)

0

tau =>


0

h^2 =>


0

# FDA4

\begin{equation}
\left\lbrace 
\begin{array}{rl}
F^1:& \operatorname{D_1}(u) + \operatorname{D_2}(v)=0 ,\\[4pt]
F^2:& \operatorname{D_t}(u) + u\operatorname{D_1}(u)+
v\operatorname{D_2}(u) + w\operatorname{D_3}(u) + 
\operatorname{D_1}(p) - \frac{1}{\mathrm{Re}} \tilde{\Delta} (u)
=0 ,\\[4pt]
F^3:& \operatorname{D_t}(v) + u\operatorname{D_1}(v)+
v\operatorname{D_2}(v) +  w\operatorname{D_3}(v) + 
\operatorname{D_2}(p) - \frac{1}{\mathrm{Re}} \tilde{\Delta} (v)
=0 ,\\[4pt]
F^4:& \operatorname{D_t}(w) + u\operatorname{D_1}(w)+
v\operatorname{D_2}(w)  +  w\operatorname{D_3}(w) +
\operatorname{D_3}(p) - \frac{1}{\mathrm{Re}} \tilde{\Delta} (w)
=0 ,\\[4pt]
F^5:& \left(\operatorname{D_1}^2 + \operatorname{D_2}^2 + \operatorname{D_3}^2\right)(p) 
    + \operatorname{D_1}(u\operatorname{D_1}(u))
    + \operatorname{D_2}(v\operatorname{D_2}(v)) + \operatorname{D_3}(w\operatorname{D_3}(w)) + {} \\
&{} +  \operatorname{D_1}(v\operatorname{D_2}(u))
    + \operatorname{D_2}(u\operatorname{D_1}(v))
- \frac{1}{\mathrm{Re}}\tilde{\Delta}(F^1)
=0 .
\end{array}
\right. 
\end{equation}

In [59]:
F1 = clip(Dx(u) + Dy(v))
F2 = clip(Dt(u) + u*Dx(u) + v*Dy(u) + Dx(p) - DD(u)/Re)
F3 = clip(Dt(v) + u*Dx(v) + v*Dy(v) + Dy(p) - DD(v)/Re)
F4 = clip(DD2(p) + Dx(u*Dx(u)) + Dy(v*Dy(v)) + Dx(v*Dy(u)) + Dy(u*Dx(v)) - (DDx(u) + DDy(v))/Re)

In [60]:
F1

[Derivative(u(t, x, y), x) + Derivative(v(t, x, y), y),
 0,
 Derivative(u(t, x, y), (x, 3))/6 + Derivative(v(t, x, y), (y, 3))/6]

In [61]:
F2

[u(t, x, y)*Derivative(u(t, x, y), x) + v(t, x, y)*Derivative(u(t, x, y), y) + Derivative(p(t, x, y), x) + Derivative(u(t, x, y), t) - Derivative(u(t, x, y), (x, 2))/Re - Derivative(u(t, x, y), (y, 2))/Re,
 Derivative(u(t, x, y), (t, 2))/2,
 u(t, x, y)*Derivative(u(t, x, y), (x, 3))/6 + v(t, x, y)*Derivative(u(t, x, y), (y, 3))/6 + Derivative(p(t, x, y), (x, 3))/6 - Derivative(u(t, x, y), (x, 4))/(12*Re) - Derivative(u(t, x, y), (y, 4))/(12*Re)]

In [62]:
F3

[u(t, x, y)*Derivative(v(t, x, y), x) + v(t, x, y)*Derivative(v(t, x, y), y) + Derivative(p(t, x, y), y) + Derivative(v(t, x, y), t) - Derivative(v(t, x, y), (x, 2))/Re - Derivative(v(t, x, y), (y, 2))/Re,
 Derivative(v(t, x, y), (t, 2))/2,
 u(t, x, y)*Derivative(v(t, x, y), (x, 3))/6 + v(t, x, y)*Derivative(v(t, x, y), (y, 3))/6 + Derivative(p(t, x, y), (y, 3))/6 - Derivative(v(t, x, y), (x, 4))/(12*Re) - Derivative(v(t, x, y), (y, 4))/(12*Re)]

In [63]:
F4

[u(t, x, y)*Derivative(u(t, x, y), (x, 2)) + u(t, x, y)*Derivative(v(t, x, y), x, y) + v(t, x, y)*Derivative(v(t, x, y), (y, 2)) + v(t, x, y)*Derivative(u(t, x, y), x, y) + Derivative(p(t, x, y), (x, 2)) + Derivative(p(t, x, y), (y, 2)) + Derivative(u(t, x, y), x)**2 + 2*Derivative(u(t, x, y), y)*Derivative(v(t, x, y), x) + Derivative(v(t, x, y), y)**2 - Derivative(u(t, x, y), (x, 3))/Re - Derivative(v(t, x, y), (y, 3))/Re - Derivative(u(t, x, y), x, (y, 2))/Re - Derivative(v(t, x, y), (x, 2), y)/Re,
 0,
 u(t, x, y)*Derivative(u(t, x, y), (x, 4))/3 + u(t, x, y)*Derivative(v(t, x, y), x, (y, 3))/6 + u(t, x, y)*Derivative(v(t, x, y), (x, 3), y)/6 + v(t, x, y)*Derivative(v(t, x, y), (y, 4))/3 + v(t, x, y)*Derivative(u(t, x, y), x, (y, 3))/6 + v(t, x, y)*Derivative(u(t, x, y), (x, 3), y)/6 + Derivative(p(t, x, y), (x, 4))/3 + Derivative(p(t, x, y), (y, 4))/3 + 5*Derivative(u(t, x, y), x)*Derivative(u(t, x, y), (x, 3))/6 + Derivative(u(t, x, y), (x, 2))**2/2 + Derivative(u(t, x, y), y)*Deri

In [64]:
f4 = NF(F4, [u.diff(x), u.diff(y, 2), v.diff(x, 2)], [F1, F2, F3], head=True)
f4 = NF(f4, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [F1, F2, F3, f4])
prn(f4)

p_{xx} + p_{yy} + 2*u_{y}*v_{x} + 2*v_{y}**2

tau =>


0

h^2 =>


4*Re**2*p_{x}*v*v_{x}/3 - Re**2*p_{yy}*u**2/6 - 2*Re**2*p_{y}*u*u_{y}/3 - Re**2*u**3*v_{xy}/6 - 5*Re**2*u**2*u_{y}*v_{x}/6 - Re**2*u**2*v*v_{yy}/6 - Re**2*u**2*v_{ty}/6 - Re**2*u**2*v_{y}**2/6 - 2*Re**2*u*u_{y}*v*v_{y}/3 - 2*Re**2*u*u_{y}*v_{t}/3 - 4*Re**2*u*v*v_{x}*v_{y}/3 + 4*Re**2*u_{t}*v*v_{x}/3 + 4*Re**2*u_{y}*v**2*v_{x}/3 - Re*p_{tyy}/6 - Re*p_{xyy}*u/6 - Re*p_{xy}*u_{y}/2 + 3*Re*p_{xy}*v_{x}/2 + 3*Re*p_{x}*v_{xy} - Re*p_{yyy}*v/6 - 5*Re*p_{yy}*v_{y}/6 + Re*p_{y}*v_{yy} + Re*u**2*v_{yyy}/6 + 2*Re*u*u_{y}*v_{yy}/3 - Re*u*v*v_{xyy}/3 - Re*u*v_{txy}/3 - 4*Re*u*v_{xy}*v_{y} - Re*u*v_{x}*v_{yy}/2 + 4*Re*u_{ty}*v_{x}/3 + 17*Re*u_{t}*v_{xy}/6 + 13*Re*u_{y}*v*v_{xy}/6 - 2*Re*u_{y}*v_{tx}/3 - 5*Re*u_{y}*v_{x}*v_{y}/6 - Re*v**2*v_{yyy}/6 - Re*v*v_{tyy}/3 + 4*Re*v*v_{xy}*v_{x}/3 - Re*v*v_{yy}*v_{y}/3 - Re*v_{tty}/6 - 7*Re*v_{ty}*v_{y}/6 + 5*Re*v_{t}*v_{yy}/6 - 5*Re*v_{y}**3/6 + p_{yyyy} + u*v_{xyyy}/3 + 5*u_{y}*v_{xyy}/2 + v*v_{yyyy}/3 + v_{tyyy}/3 + 3*v_{xyy}*v_{x}/2 + 2*v_{xy}**2 + 14*v_{

In [65]:
prn([f.subs(TD).doit().simplify() for f in f4])

0

tau =>


0

h^2 =>


(sin(x)**2 + sin(y)**2 - 1)*exp(-4*t/Re)

In [66]:
f1 = NF(F1, [u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [F2, F3, f4], head=True)
f1 = NF(f1, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [f1, F2, F3, f4])
prn(f1)

u_{x} + v_{y}

tau =>


0

h^2 =>


-Re*p_{yy}/6 - Re*u*v_{xy}/6 - Re*u_{y}*v_{x}/6 - Re*v*v_{yy}/6 - Re*v_{ty}/6 - Re*v_{y}**2/6 + v_{yyy}/3

In [67]:
prn([f.subs(TD).doit().simplify() for f in f1])

0

tau =>


0

h^2 =>


0

In [68]:
f2 = NF(F2, [u.diff(x), v.diff(x, 2), p.diff(x, 2)], [f1, F3, f4], head=True)
f2 = NF(f2, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [f1, f2, F3, f4])
prn(f2)

p_{x} - u*v_{y} + u_{t} + u_{y}*v - u_{yy}/Re + v_{xy}/Re

tau =>


u_{tt}/2

h^2 =>


Re**2*p_{x}*v**2/12 - Re**2*u*v**2*v_{y}/12 + Re**2*u_{t}*v**2/12 + Re**2*u_{y}*v**3/12 - Re*p_{tx}/12 + Re*p_{xy}*v/12 - Re*p_{x}*v_{y}/12 - Re*p_{yy}*u/12 - 5*Re*p_{y}*u_{y}/12 - Re*u**2*v_{xy}/12 - Re*u*u_{y}*v_{x}/2 - Re*u*v*v_{yy}/6 - Re*u_{tt}/12 - Re*u_{y}*v*v_{y}/2 - Re*u_{y}*v_{t}/2 + Re*v**2*v_{xy}/12 - p_{xyy}/3 + u_{y}*v_{yy}/2 - v_{txy}/6 - 5*v_{xy}*v_{y}/6 + v_{x}*v_{yy}/3 + v_{xyyy}/(6*Re)

In [69]:
prn([f.subs(TD).doit().simplify() for f in f2])

0

tau =>


-2*exp(-2*t/Re)*sin(y)*cos(x)/Re**2

h^2 =>


(-3*Re*sin(x) + exp(2*t/Re)*sin(y))*exp(-4*t/Re)*cos(x)/(6*Re)

In [70]:
f3 = NF(F3, [u.diff(x), u.diff(y, 2), p.diff(x, 2)], [f1, f2, f4], head=True)
f3 = NF(f3, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [f1, f2, f3, f4])
prn(f3)

p_{y} + u*v_{x} + v*v_{y} + v_{t} - v_{xx}/Re - v_{yy}/Re

tau =>


v_{tt}/2

h^2 =>


Re**2*p_{y}*u**2/12 + Re**2*u**3*v_{x}/12 + Re**2*u**2*v*v_{y}/12 + Re**2*u**2*v_{t}/12 - Re*p_{ty}/12 + Re*p_{xy}*u/12 + Re*p_{x}*v_{x}/4 - Re*p_{yy}*v/12 + Re*p_{y}*v_{y}/12 - Re*u**2*v_{yy}/12 - Re*u*v_{x}*v_{y}/6 + Re*u_{t}*v_{x}/6 + Re*u_{y}*v*v_{x}/6 - Re*v**2*v_{yy}/12 - Re*v*v_{ty}/6 - Re*v_{tt}/12 + p_{yyy}/3 + u_{y}*v_{xy}/3 + v*v_{yyy}/3 + v_{tyy}/6 + v_{xy}*v_{x}/6 + v_{yy}*v_{y}/2 - v_{yyyy}/(6*Re)

In [71]:
prn([f.subs(TD).doit().simplify() for f in f3])

0

tau =>


2*exp(-2*t/Re)*sin(x)*cos(y)/Re**2

h^2 =>


-(3*Re*sin(y) + exp(2*t/Re)*sin(x))*exp(-4*t/Re)*cos(y)/(6*Re)

In [72]:
spoly = [a.diff(x) + b.diff(y) for a, b in zip(f2, f3)]
spoly = NF(spoly, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [f1, f2, f3, f4], head=True)
prn(spoly)

0

tau =>


0

h^2 =>


-Re**2*p_{x}*v*v_{x}/3 + Re**2*p_{yy}*u**2/6 + Re**2*p_{y}*u*u_{y}/3 + Re**2*u**3*v_{xy}/6 + Re**2*u**2*u_{y}*v_{x}/2 + Re**2*u**2*v*v_{yy}/6 + Re**2*u**2*v_{ty}/6 + Re**2*u**2*v_{y}**2/6 + Re**2*u*u_{y}*v*v_{y}/3 + Re**2*u*u_{y}*v_{t}/3 + Re**2*u*v*v_{x}*v_{y}/3 - Re**2*u_{t}*v*v_{x}/3 - Re**2*u_{y}*v**2*v_{x}/3 + Re*p_{tyy}/6 + Re*p_{xyy}*u/6 + Re*p_{xy}*u_{y}/6 - Re*p_{xy}*v_{x}/2 - 7*Re*p_{x}*v_{xy}/6 + Re*p_{yyy}*v/6 - Re*p_{y}*v_{yy}/6 - Re*u**2*v_{yyy}/6 - Re*u*u_{y}*v_{yy}/3 + Re*u*v*v_{xyy}/3 + Re*u*v_{txy}/3 + 4*Re*u*v_{xy}*v_{y}/3 + Re*u*v_{x}*v_{yy}/3 - Re*u_{ty}*v_{x}/3 - Re*u_{t}*v_{xy} - 2*Re*u_{y}*v*v_{xy}/3 + Re*u_{y}*v_{tx}/3 + Re*v**2*v_{yyy}/6 + Re*v*v_{tyy}/3 - Re*v*v_{xy}*v_{x}/3 + Re*v*v_{yy}*v_{y}/3 + Re*v_{tty}/6 + Re*v_{ty}*v_{y}/3 - p_{yyyy}/3 - u*v_{xyyy}/3 - u_{y}*v_{xyy} - v*v_{yyyy}/3 - v_{tyyy}/3 - v_{xyy}*v_{x}/3 - v_{xy}**2 - 4*v_{yyy}*v_{y}/3 - v_{yy}**2

In [73]:
prn([f.subs(TD).doit().simplify() for f in spoly])

0

tau =>


0

h^2 =>


0

In [74]:
prn([f.subs(KF).doit().simplify() for f in spoly])

0

tau =>


0

h^2 =>


(Re - sqrt(Re**2 + 16*pi**2))*(Re**3 - Re**2*sqrt(Re**2 + 16*pi**2) + 16*pi**2*Re - 8*pi**2*sqrt(Re**2 + 16*pi**2))*exp(x*(Re - sqrt(Re**2 + 16*pi**2))/2)*cos(2*pi*y)/24

# MAC

In [75]:
p12 = Rational(1, 2)

def F(j):
    return T(u, 0, j, 0) + tau/h*(\
        (T(u, 0, j+1, 0) - 2*T(u, 0, j, 0) + T(u, 0, j-1, 0))/(Re*h) +\
        (T(u, 0, j, 1) -   2*T(u, 0, j, 0) + T(u, 0, j, -1))/(Re*h) -\
        ((T(u, 0, j+1, 0) + T(u, 0, j, 0))**2 - (T(u, 0, j, 0) + T(u, 0, j-1, 0))**2)/4 -\
        ((T(u, 0, j, 1) + T(u, 0, j, 0)) *(T(v, 0, j+1, p12)  + T(v, 0, j-1, p12)) -\
         (T(u, 0, j, 0) + T(u, 0, j, -1))*(T(v, 0, j+1, -p12) + T(v, 0, j-1, -p12)))/4)

def G(k):
    return T(v, 0, 0, k) + tau/h*(\
        (T(v, 0, 1, k) -   2*T(v, 0, 0, k) + T(v, 0, -1, k))/(Re*h) +\
        (T(v, 0, 0, k+1) - 2*T(v, 0, 0, k) + T(v, 0, 0, k-1))/(Re*h) -\
        ((T(u, 0, p12, k+1)  + T(u, 0, p12, k-1))* (T(v, 0, 1, k) + T(v, 0, 0, k)) -\
         (T(u, 0, -p12, k+1) + T(u, 0, -p12, k-1))*(T(v, 0, 0, k) + T(v, 0, -1, k)))/4 -\
        ((T(v, 0, 0, k+1) + T(v, 0, 0, k))**2 - (T(v, 0, 0, k) + T(v, 0, 0, k-1))**2)/4)

<img src="./F17.13.png"/>
<img src="./F17.8.png"/>
<img src="./F17.10.png"/>

In [76]:
F4 = clip((T(p, 1, 1, 0) - 2*T(p, 1, 0, 0) + T(p, 1, -1, 0))/h**2 +\
          (T(p, 1, 0, 1) - 2*T(p, 1, 0, 0) + T(p, 1, 0, -1))/h**2 -\
         ((F(p12) - F(-p12)) + (G(p12) - G(-p12)))/(tau*h))
prn(F4)

p_{xx} + p_{yy} + 2*u*u_{xx} + 2*u*v_{xy} + 2*u_{xy}*v + 2*u_{x}**2 + 2*u_{x}*v_{y} + 2*u_{y}*v_{x} + 2*v*v_{yy} + 2*v_{y}**2 - u_{xxx}/Re - u_{xyy}/Re - v_{xxy}/Re - v_{yyy}/Re

tau =>


p_{txx} + p_{tyy}

h^2 =>


p_{xxxx}/12 + p_{yyyy}/12 + 5*u*u_{xxxx}/12 + 17*u*v_{xxxy}/24 + u*v_{xyyy}/12 + u_{xxxy}*v/12 + 7*u_{xxx}*u_{x}/6 + u_{xxx}*v_{y}/12 + u_{xxy}*v_{x}/4 + 3*u_{xx}**2/4 + u_{xx}*v_{xy}/4 + 17*u_{xyyy}*v/24 + 7*u_{xyy}*v_{y}/8 + 7*u_{xy}*v_{xx}/8 + u_{xy}*v_{yy}/4 + 7*u_{x}*v_{xxy}/8 + u_{x}*v_{yyy}/12 + 17*u_{yyy}*v_{x}/24 + 7*u_{yy}*v_{xy}/8 + 17*u_{y}*v_{xxx}/24 + u_{y}*v_{xyy}/4 + 5*v*v_{yyyy}/12 + 7*v_{yyy}*v_{y}/6 + 3*v_{yy}**2/4

In [77]:
F2 = clip((T(u, 1, p12, 0) - F(p12))/tau +\
          (T(p, 1, 1, 0) - T(p, 1, 0, 0))/h)
prn(F2)

p_{x} + 2*u*u_{x} + u*v_{y} + u_{t} + u_{y}*v - u_{xx}/Re - u_{yy}/Re

tau =>


p_{tx} + u_{tt}/2

h^2 =>


p_{xxx}/6 + 7*u*u_{xxx}/12 + 5*u*v_{xxy}/8 + u*v_{yyy}/24 + u_{txx}/8 + u_{xxy}*v/8 + 5*u_{xx}*u_{x}/4 + u_{xx}*v_{y}/8 + u_{xy}*v_{x}/4 + u_{x}*v_{xy}/4 + u_{yyy}*v/6 + u_{yy}*v_{y}/4 + 5*u_{y}*v_{xx}/8 + u_{y}*v_{yy}/8 - 5*u_{xxxx}/(24*Re) - u_{xxyy}/(8*Re) - u_{yyyy}/(12*Re)

In [78]:
F3 = clip((T(v, 1, 0, p12) - G(p12))/tau +\
          (T(p, 1, 0, 1) - T(p, 1, 0, 0))/h)
prn(F3)

p_{y} + u*v_{x} + u_{x}*v + 2*v*v_{y} + v_{t} - v_{xx}/Re - v_{yy}/Re

tau =>


p_{ty} + v_{tt}/2

h^2 =>


p_{yyy}/6 + u*v_{xxx}/6 + u*v_{xyy}/8 + u_{xxx}*v/24 + u_{xx}*v_{x}/8 + 5*u_{xyy}*v/8 + u_{xy}*v_{y}/4 + u_{x}*v_{xx}/4 + u_{x}*v_{yy}/8 + 5*u_{yy}*v_{x}/8 + u_{y}*v_{xy}/4 + 7*v*v_{yyy}/12 + v_{tyy}/8 + 5*v_{yy}*v_{y}/4 - v_{xxxx}/(12*Re) - v_{xxyy}/(8*Re) - 5*v_{yyyy}/(24*Re)

In [79]:
F1 = clip(((T(u, 0, p12, 0) - T(u, 1, -p12, 0)) +\
           (T(v, 0, 0, p12) - T(v, 1, 0, -p12)))/h)
prn(F1)

u_{x} + v_{y}

tau =>


u_{tx}/2 + v_{ty}/2

h^2 =>


u_{xxx}/24 + v_{yyy}/24

In [80]:
f4 = NF(F4, [u.diff(x), u.diff(y, 2), v.diff(x, 2)], [F1, F2, F3], head=True)
f4 = NF(f4, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [F1, F2, F3, f4])
prn(f4)

p_{xx} + p_{yy} + 2*u_{y}*v_{x} + 2*v_{y}**2

tau =>


-2*u_{ty}*v_{x} - 2*u_{y}*v_{tx} - 4*v_{ty}*v_{y}

h^2 =>


23*Re**2*p_{x}*v*v_{x}/24 + Re**2*p_{yy}*u**2/3 + 5*Re**2*p_{y}*u*u_{y}/6 + Re**2*u**3*v_{xy}/3 + 7*Re**2*u**2*u_{y}*v_{x}/6 + Re**2*u**2*v*v_{yy}/3 + Re**2*u**2*v_{ty}/3 + Re**2*u**2*v_{y}**2/3 + 5*Re**2*u*u_{y}*v*v_{y}/6 + 5*Re**2*u*u_{y}*v_{t}/6 - 23*Re**2*u*v*v_{x}*v_{y}/24 + 23*Re**2*u_{t}*v*v_{x}/24 + 23*Re**2*u_{y}*v**2*v_{x}/24 - Re*p_{tyy}/24 + Re*p_{xyy}*u/3 + Re*p_{xy}*u_{y}/2 + Re*p_{xy}*v_{x} + 3*Re*p_{x}*v_{xy}/2 - Re*p_{yyy}*v/24 - 5*Re*p_{yy}*v_{y}/24 - Re*p_{y}*v_{yy}/2 - Re*u**2*v_{yyy}/3 - 5*Re*u*u_{y}*v_{yy}/6 + 7*Re*u*v*v_{xyy}/24 + 7*Re*u*v_{txy}/24 - 11*Re*u*v_{xy}*v_{y}/8 - 3*Re*u*v_{x}*v_{yy}/2 + 23*Re*u_{ty}*v_{x}/24 + 35*Re*u_{t}*v_{xy}/24 + 23*Re*u_{y}*v*v_{xy}/12 + 11*Re*u_{y}*v_{tx}/24 - 5*Re*u_{y}*v_{x}*v_{y}/24 - Re*v**2*v_{yyy}/24 - Re*v*v_{tyy}/12 + 23*Re*v*v_{xy}*v_{x}/24 - 5*Re*v*v_{yy}*v_{y}/6 - Re*v_{tty}/24 - 7*Re*v_{ty}*v_{y}/24 - 13*Re*v_{t}*v_{yy}/24 - 5*Re*v_{y}**3/24 + p_{yyyy}/4 - 7*u*v_{xyyy}/24 + 2*u_{tty}*v_{x} + 4*u_{ty}*v_{tx} + 2*u_{y}

In [81]:
prn([f.subs(TD).doit().simplify() for f in f4])

0

tau =>


-(4*cos(2*x) + 4*cos(2*y))*exp(-4*t/Re)/Re

h^2 =>


(12*Re**2*sin(x)**2*sin(y)**2 - 8*Re**2*sin(x)**2 - 8*Re**2*sin(y)**2 + 5*Re**2 - 64*cos(2*x) - 64*cos(2*y))*exp(-4*t/Re)/(4*Re**2)

In [82]:
f1 = NF(F1, [u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [F2, F3, f4], head=True)
f1 = NF(f1, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [f1, F2, F3, f4])
prn(f1)

u_{x} + v_{y}

tau =>


0

h^2 =>


-Re*p_{yy}/24 - Re*u*v_{xy}/24 - Re*u_{y}*v_{x}/24 - Re*v*v_{yy}/24 - Re*v_{ty}/24 - Re*v_{y}**2/24 + v_{yyy}/12

In [83]:
prn([f.subs(TD).doit().simplify() for f in f1])

0

tau =>


0

h^2 =>


0

In [84]:
f2 = NF(F2, [u.diff(x), v.diff(x, 2), p.diff(x, 2)], [f1, F3, f4], head=True)
f2 = NF(f2, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [f1, f2, F3, f4])
prn(f2)

p_{x} - u*v_{y} + u_{t} + u_{y}*v - u_{yy}/Re + v_{xy}/Re

tau =>


p_{tx} + u_{tt}/2

h^2 =>


Re**2*p_{x}*v**2/12 - Re**2*u*v**2*v_{y}/12 + Re**2*u_{t}*v**2/12 + Re**2*u_{y}*v**3/12 - Re*p_{tx}/12 + Re*p_{xy}*v/12 + Re*p_{x}*v_{y}/6 + 7*Re*p_{yy}*u/24 + 11*Re*p_{y}*u_{y}/24 + 7*Re*u**2*v_{xy}/24 + 3*Re*u*u_{y}*v_{x}/4 + 5*Re*u*v*v_{yy}/24 + 3*Re*u*v_{ty}/8 + Re*u*v_{y}**2/8 - Re*u_{tt}/12 + Re*u_{t}*v_{y}/4 + 5*Re*u_{y}*v*v_{y}/8 + 3*Re*u_{y}*v_{t}/8 + Re*v**2*v_{xy}/12 - p_{xyy}/12 - u*v_{yyy}/4 - u_{y}*v_{yy}/4 + v*v_{xyy}/8 - v_{txy}/24 + 13*v_{xy}*v_{y}/24 + v_{x}*v_{yy}/12 - v_{xyyy}/(12*Re)

In [85]:
prn([f.subs(TD).doit().simplify() for f in f2])

0

tau =>


-2*(2*Re*sin(x) + exp(2*t/Re)*sin(y))*exp(-4*t/Re)*cos(x)/Re**2

h^2 =>


(3*Re*(5 - 6*sin(y)**2)*sin(x) + 4*exp(2*t/Re)*sin(y))*exp(-4*t/Re)*cos(x)/(24*Re)

In [86]:
f3 = NF(F3, [u.diff(x), u.diff(y, 2), p.diff(x, 2)], [f1, f2, f4], head=True)
f3 = NF(f3, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [f1, f2, f3, f4])
prn(f3)

p_{y} + u*v_{x} + v*v_{y} + v_{t} - v_{xx}/Re - v_{yy}/Re

tau =>


p_{ty} + v_{tt}/2

h^2 =>


Re**2*p_{y}*u**2/12 + Re**2*u**3*v_{x}/12 + Re**2*u**2*v*v_{y}/12 + Re**2*u**2*v_{t}/12 - Re*p_{ty}/12 + Re*p_{xy}*u/12 + 3*Re*p_{x}*v_{x}/4 - Re*p_{yy}*v/12 - Re*p_{y}*v_{y}/6 - Re*u**2*v_{yy}/12 - 11*Re*u*v_{x}*v_{y}/12 + 2*Re*u_{t}*v_{x}/3 + 2*Re*u_{y}*v*v_{x}/3 - Re*v**2*v_{yy}/12 - Re*v*v_{ty}/6 - Re*v*v_{y}**2/4 - Re*v_{tt}/12 - Re*v_{t}*v_{y}/4 + 5*p_{yyy}/24 + u_{y}*v_{xy}/3 - v*v_{yyy}/24 + v_{tyy}/6 + 13*v_{xy}*v_{x}/24 + 5*v_{yy}*v_{y}/4 - v_{yyyy}/(6*Re)

In [87]:
prn([f.subs(TD).doit().simplify() for f in f3])

0

tau =>


2*(-2*Re*sin(y) + exp(2*t/Re)*sin(x))*exp(-4*t/Re)*cos(y)/Re**2

h^2 =>


(3*Re*(6*cos(x)**2 - 1)*sin(y) - 4*exp(2*t/Re)*sin(x))*exp(-4*t/Re)*cos(y)/(24*Re)

In [88]:
spoly = [a.diff(x) + b.diff(y) for a, b in zip(f2, f3)]
spoly = NF(spoly, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [f1, f2, f3, f4], head=True)
prn(spoly)

0

tau =>


0

h^2 =>


-Re**2*p_{x}*v*v_{x}/12 + Re**2*p_{yy}*u**2/24 + Re**2*p_{y}*u*u_{y}/12 + Re**2*u**3*v_{xy}/24 + Re**2*u**2*u_{y}*v_{x}/8 + Re**2*u**2*v*v_{yy}/24 + Re**2*u**2*v_{ty}/24 + Re**2*u**2*v_{y}**2/24 + Re**2*u*u_{y}*v*v_{y}/12 + Re**2*u*u_{y}*v_{t}/12 + Re**2*u*v*v_{x}*v_{y}/12 - Re**2*u_{t}*v*v_{x}/12 - Re**2*u_{y}*v**2*v_{x}/12 + Re*p_{tyy}/24 + Re*p_{xyy}*u/24 + Re*p_{xy}*u_{y}/24 - Re*p_{xy}*v_{x}/8 - 7*Re*p_{x}*v_{xy}/24 + Re*p_{yyy}*v/24 - Re*p_{y}*v_{yy}/24 - Re*u**2*v_{yyy}/24 - Re*u*u_{y}*v_{yy}/12 + Re*u*v*v_{xyy}/12 + Re*u*v_{txy}/12 + Re*u*v_{xy}*v_{y}/3 + Re*u*v_{x}*v_{yy}/12 - Re*u_{ty}*v_{x}/12 - Re*u_{t}*v_{xy}/4 - Re*u_{y}*v*v_{xy}/6 + Re*u_{y}*v_{tx}/12 + Re*v**2*v_{yyy}/24 + Re*v*v_{tyy}/12 - Re*v*v_{xy}*v_{x}/12 + Re*v*v_{yy}*v_{y}/12 + Re*v_{tty}/24 + Re*v_{ty}*v_{y}/12 - p_{yyyy}/12 - u*v_{xyyy}/12 - u_{y}*v_{xyy}/4 - v*v_{yyyy}/12 - v_{tyyy}/12 - v_{xyy}*v_{x}/12 - v_{xy}**2/4 - v_{yyy}*v_{y}/3 - v_{yy}**2/4

In [89]:
prn([f.subs(TD).doit().simplify() for f in spoly])

0

tau =>


0

h^2 =>


0

In [90]:
prn([f.subs(KF).doit().simplify() for f in spoly])

0

tau =>


0

h^2 =>


(Re - sqrt(Re**2 + 16*pi**2))*(Re**3 - Re**2*sqrt(Re**2 + 16*pi**2) + 16*pi**2*Re - 8*pi**2*sqrt(Re**2 + 16*pi**2))*exp(x*(Re - sqrt(Re**2 + 16*pi**2))/2)*cos(2*pi*y)/96

# Неявная

$\operatorname{I_t} = \dfrac{1 + \sigma_t}{2}$
\begin{equation}
\left\lbrace 
\begin{array}{rl}
F^1:& \operatorname{D_1}(u) + \operatorname{D_2}(v)=0 ,\\[4pt]
F^2:& \operatorname{D_t}(u) + \operatorname{I_t}\left(\operatorname{D_1}(u^2)+
\operatorname{D_2}(vu) + 
\operatorname{D_1}(p) - \frac{1}{\mathrm{Re}} \tilde{\Delta} (u) \right)
=0 ,\\[4pt]
F^3:& \operatorname{D_t}(v) + \operatorname{I_t}\left(\operatorname{D_1}(uv)+
\operatorname{D_2}(v^2) +
\operatorname{D_2}(p) - \frac{1}{\mathrm{Re}} \tilde{\Delta} (v) \right)
=0 ,\\[4pt]
F^4:& \left(\operatorname{D_1}^2 + \operatorname{D_2}^2\right)(p) + \operatorname{D_1}^2(u^2) + \operatorname{D_2}^2(v^2) + {} \\
&{} + 2\operatorname{D_1}\operatorname{D_2}(uv)
=0 .
\end{array}
\right. 
\end{equation}

In [91]:
pda_p = -Rational(1, 2), Rational(0, 1), Rational(0, 1)
def Dt(a):
    return (T(a, 1, 0, 0) - T(a, 0, 0, 0))/(tau)
def Dx(a):
    return ((T(a, 1, 1, 0) - T(a, 1, -1, 0)) +\
            (T(a, 0, 1, 0) - T(a, 0, -1, 0)))/(4*h)
def Dy(a):
    return ((T(a, 1, 0, 1) - T(a, 1, 0, -1)) +\
            (T(a, 0, 0, 1) - T(a, 0, 0, -1)))/(4*h)

def DD(a):
    return ((T(a, 1, 1, 0) +  T(a, 1, 0, 1) +\
                 -4*T(a, 1, 0, 0) +\
             T(a, 1, -1, 0) + T(a, 1, 0, -1)) +\
            (T(a, 0, 1, 0) +  T(a, 0, 0, 1) +\
                 -4*T(a, 0, 0, 0) +\
             T(a, 0, -1, 0) + T(a, 0, 0, -1)))/(2*h**2)

def DD2(a):
    return ((T(a, 1, 2, 0) +  T(a, 1, 0, 2) +\
                 -4*T(a, 1, 0, 0) +\
             T(a, 1, -2, 0) + T(a, 1, 0, -2)) +\
            (T(a, 0, 2, 0) +  T(a, 0, 0, 2) +\
                 -4*T(a, 0, 0, 0) +\
             T(a, 0, -2, 0) + T(a, 0, 0, -2)))/(8*h**2)

def Dxx(a):
    return ((T(a, 1, 2, 0) - 2*T(a, 1, 0, 0) + T(a, 1, -2, 0)) +\
            (T(a, 0, 2, 0) - 2*T(a, 0, 0, 0) + T(a, 0, -2, 0)))/(8*h**2)
def Dxy(a):
    return ((T(a, 1, 1, 1) -  T(a, 1, 1, -1) -\
             T(a, 1, -1, 1) + T(a, 1, -1, -1)) +\
            (T(a, 0, 1, 1) - T(a, 0, 1, -1) -\
            T(a, 0, -1, 1) + T(a, 0, -1, -1)))/(8*h**2)
def Dyy(a):
    return ((T(a, 1, 0, 2) - 2*T(a, 1, 0, 0) + T(a, 1, 0, -2)) +
            (T(a, 0, 0, 2) - 2*T(a, 0, 0, 0) + T(a, 0, 0, -2)))/(8*h**2)

def clip2(f):
    f = f.expand()
    return [f.coeff(tau, 0).coeff(h, 0),\
            f.coeff(h, 0).coeff(tau, 2),\
            f.coeff(tau, 0).coeff(h, 2)]
def prn2(a):
    display(a[0])
    print("tau^2 =>")
    display(a[1])
    print("h^2 =>")
    display(a[2])

In [92]:
F1 = clip2(Dx(u) + Dy(v))
F2 = clip2(Dt(u) + Dx(u**2) + Dy(v*u) +  Dx(p) - DD(u)/Re)
F3 = clip2(Dt(v) + Dx(u*v) +  Dy(v**2) + Dy(p) - DD(v)/Re)
F4 = clip2(DD2(p) + Dxx(u**2) + Dyy(v**2) + 2*Dxy(u*v))

In [93]:
prn2(F1)

Derivative(u(t, x, y), x) + Derivative(v(t, x, y), y)

tau^2 =>


Derivative(u(t, x, y), (t, 2), x)/8 + Derivative(v(t, x, y), (t, 2), y)/8

h^2 =>


Derivative(u(t, x, y), (x, 3))/6 + Derivative(v(t, x, y), (y, 3))/6

In [94]:
prn2(F2)

2*u(t, x, y)*Derivative(u(t, x, y), x) + u(t, x, y)*Derivative(v(t, x, y), y) + v(t, x, y)*Derivative(u(t, x, y), y) + Derivative(p(t, x, y), x) + Derivative(u(t, x, y), t) - Derivative(u(t, x, y), (x, 2))/Re - Derivative(u(t, x, y), (y, 2))/Re

tau^2 =>


u(t, x, y)*Derivative(u(t, x, y), (t, 2), x)/4 + u(t, x, y)*Derivative(v(t, x, y), (t, 2), y)/8 + v(t, x, y)*Derivative(u(t, x, y), (t, 2), y)/8 + Derivative(u(t, x, y), t)*Derivative(u(t, x, y), t, x)/2 + Derivative(u(t, x, y), t)*Derivative(v(t, x, y), t, y)/4 + Derivative(u(t, x, y), (t, 2))*Derivative(u(t, x, y), x)/4 + Derivative(u(t, x, y), (t, 2))*Derivative(v(t, x, y), y)/8 + Derivative(u(t, x, y), (t, 3))/24 + Derivative(u(t, x, y), y)*Derivative(v(t, x, y), (t, 2))/8 + Derivative(v(t, x, y), t)*Derivative(u(t, x, y), t, y)/4 + Derivative(p(t, x, y), (t, 2), x)/8 - Derivative(u(t, x, y), (t, 2), (x, 2))/(8*Re) - Derivative(u(t, x, y), (t, 2), (y, 2))/(8*Re)

h^2 =>


u(t, x, y)*Derivative(u(t, x, y), (x, 3))/3 + u(t, x, y)*Derivative(v(t, x, y), (y, 3))/6 + v(t, x, y)*Derivative(u(t, x, y), (y, 3))/6 + Derivative(p(t, x, y), (x, 3))/6 + Derivative(u(t, x, y), x)*Derivative(u(t, x, y), (x, 2)) + Derivative(u(t, x, y), y)*Derivative(v(t, x, y), (y, 2))/2 + Derivative(u(t, x, y), (y, 2))*Derivative(v(t, x, y), y)/2 - Derivative(u(t, x, y), (x, 4))/(12*Re) - Derivative(u(t, x, y), (y, 4))/(12*Re)

In [95]:
prn2(F3)

u(t, x, y)*Derivative(v(t, x, y), x) + v(t, x, y)*Derivative(u(t, x, y), x) + 2*v(t, x, y)*Derivative(v(t, x, y), y) + Derivative(p(t, x, y), y) + Derivative(v(t, x, y), t) - Derivative(v(t, x, y), (x, 2))/Re - Derivative(v(t, x, y), (y, 2))/Re

tau^2 =>


u(t, x, y)*Derivative(v(t, x, y), (t, 2), x)/8 + v(t, x, y)*Derivative(u(t, x, y), (t, 2), x)/8 + v(t, x, y)*Derivative(v(t, x, y), (t, 2), y)/4 + Derivative(u(t, x, y), t)*Derivative(v(t, x, y), t, x)/4 + Derivative(u(t, x, y), (t, 2))*Derivative(v(t, x, y), x)/8 + Derivative(u(t, x, y), x)*Derivative(v(t, x, y), (t, 2))/8 + Derivative(v(t, x, y), t)*Derivative(u(t, x, y), t, x)/4 + Derivative(v(t, x, y), t)*Derivative(v(t, x, y), t, y)/2 + Derivative(v(t, x, y), (t, 2))*Derivative(v(t, x, y), y)/4 + Derivative(v(t, x, y), (t, 3))/24 + Derivative(p(t, x, y), (t, 2), y)/8 - Derivative(v(t, x, y), (t, 2), (x, 2))/(8*Re) - Derivative(v(t, x, y), (t, 2), (y, 2))/(8*Re)

h^2 =>


u(t, x, y)*Derivative(v(t, x, y), (x, 3))/6 + v(t, x, y)*Derivative(u(t, x, y), (x, 3))/6 + v(t, x, y)*Derivative(v(t, x, y), (y, 3))/3 + Derivative(p(t, x, y), (y, 3))/6 + Derivative(u(t, x, y), x)*Derivative(v(t, x, y), (x, 2))/2 + Derivative(u(t, x, y), (x, 2))*Derivative(v(t, x, y), x)/2 + Derivative(v(t, x, y), y)*Derivative(v(t, x, y), (y, 2)) - Derivative(v(t, x, y), (x, 4))/(12*Re) - Derivative(v(t, x, y), (y, 4))/(12*Re)

In [96]:
prn2(F4)

2*u(t, x, y)*Derivative(u(t, x, y), (x, 2)) + 2*u(t, x, y)*Derivative(v(t, x, y), x, y) + 2*v(t, x, y)*Derivative(v(t, x, y), (y, 2)) + 2*v(t, x, y)*Derivative(u(t, x, y), x, y) + Derivative(p(t, x, y), (x, 2)) + Derivative(p(t, x, y), (y, 2)) + 2*Derivative(u(t, x, y), x)**2 + 2*Derivative(u(t, x, y), x)*Derivative(v(t, x, y), y) + 2*Derivative(u(t, x, y), y)*Derivative(v(t, x, y), x) + 2*Derivative(v(t, x, y), y)**2

tau^2 =>


u(t, x, y)*Derivative(u(t, x, y), (t, 2), (x, 2))/4 + u(t, x, y)*Derivative(v(t, x, y), (t, 2), x, y)/4 + v(t, x, y)*Derivative(v(t, x, y), (t, 2), (y, 2))/4 + v(t, x, y)*Derivative(u(t, x, y), (t, 2), x, y)/4 + Derivative(u(t, x, y), t)*Derivative(u(t, x, y), t, (x, 2))/2 + Derivative(u(t, x, y), t)*Derivative(v(t, x, y), t, x, y)/2 + Derivative(u(t, x, y), (t, 2))*Derivative(u(t, x, y), (x, 2))/4 + Derivative(u(t, x, y), (t, 2))*Derivative(v(t, x, y), x, y)/4 + Derivative(u(t, x, y), x)*Derivative(u(t, x, y), (t, 2), x)/2 + Derivative(u(t, x, y), x)*Derivative(v(t, x, y), (t, 2), y)/4 + Derivative(u(t, x, y), y)*Derivative(v(t, x, y), (t, 2), x)/4 + Derivative(v(t, x, y), t)*Derivative(v(t, x, y), t, (y, 2))/2 + Derivative(v(t, x, y), t)*Derivative(u(t, x, y), t, x, y)/2 + Derivative(v(t, x, y), (t, 2))*Derivative(v(t, x, y), (y, 2))/4 + Derivative(v(t, x, y), (t, 2))*Derivative(u(t, x, y), x, y)/4 + Derivative(v(t, x, y), x)*Derivative(u(t, x, y), (t, 2), y)/4 + Derivative(v(t, x, y

h^2 =>


2*u(t, x, y)*Derivative(u(t, x, y), (x, 4))/3 + u(t, x, y)*Derivative(v(t, x, y), x, (y, 3))/3 + u(t, x, y)*Derivative(v(t, x, y), (x, 3), y)/3 + 2*v(t, x, y)*Derivative(v(t, x, y), (y, 4))/3 + v(t, x, y)*Derivative(u(t, x, y), x, (y, 3))/3 + v(t, x, y)*Derivative(u(t, x, y), (x, 3), y)/3 + Derivative(p(t, x, y), (x, 4))/3 + Derivative(p(t, x, y), (y, 4))/3 + 8*Derivative(u(t, x, y), x)*Derivative(u(t, x, y), (x, 3))/3 + Derivative(u(t, x, y), x)*Derivative(v(t, x, y), (y, 3))/3 + Derivative(u(t, x, y), x)*Derivative(v(t, x, y), (x, 2), y) + 2*Derivative(u(t, x, y), (x, 2))**2 + Derivative(u(t, x, y), (x, 2))*Derivative(v(t, x, y), x, y) + Derivative(u(t, x, y), (x, 3))*Derivative(v(t, x, y), y)/3 + Derivative(u(t, x, y), y)*Derivative(v(t, x, y), (x, 3))/3 + Derivative(u(t, x, y), y)*Derivative(v(t, x, y), x, (y, 2)) + Derivative(u(t, x, y), (y, 2))*Derivative(v(t, x, y), x, y) + Derivative(u(t, x, y), (y, 3))*Derivative(v(t, x, y), x)/3 + Derivative(v(t, x, y), x)*Derivative(u(t, x, 

In [97]:
f4 = NF(F4, [u.diff(x), u.diff(y, 2), v.diff(x, 2)], [F1, F2, F3], head=True)
f4 = NF(f4, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [F1, F2, F3, f4])
prn2(f4)

Derivative(p(t, x, y), (x, 2)) + Derivative(p(t, x, y), (y, 2)) + 2*Derivative(u(t, x, y), y)*Derivative(v(t, x, y), x) + 2*Derivative(v(t, x, y), y)**2

tau^2 =>


0

h^2 =>


-Re**2*u(t, x, y)**2*Derivative(u(t, x, y), y)*Derivative(v(t, x, y), x)/3 - Re**2*u(t, x, y)*v(t, x, y)*Derivative(u(t, x, y), y)*Derivative(v(t, x, y), y)/3 - Re**2*u(t, x, y)*v(t, x, y)*Derivative(v(t, x, y), x)*Derivative(v(t, x, y), y) - Re**2*u(t, x, y)*Derivative(p(t, x, y), y)*Derivative(u(t, x, y), y)/3 - Re**2*u(t, x, y)*Derivative(u(t, x, y), y)*Derivative(v(t, x, y), t)/3 + Re**2*v(t, x, y)**2*Derivative(u(t, x, y), y)*Derivative(v(t, x, y), x) + Re**2*v(t, x, y)*Derivative(p(t, x, y), x)*Derivative(v(t, x, y), x) + Re**2*v(t, x, y)*Derivative(u(t, x, y), t)*Derivative(v(t, x, y), x) + Re*u(t, x, y)*Derivative(u(t, x, y), y)*Derivative(v(t, x, y), (y, 2))/3 - 2*Re*u(t, x, y)*Derivative(v(t, x, y), x)*Derivative(v(t, x, y), (y, 2))/3 - 8*Re*u(t, x, y)*Derivative(v(t, x, y), y)*Derivative(v(t, x, y), x, y)/3 + 2*Re*v(t, x, y)*Derivative(u(t, x, y), y)*Derivative(v(t, x, y), x, y) + Re*v(t, x, y)*Derivative(v(t, x, y), x)*Derivative(v(t, x, y), x, y) + 7*Re*Derivative(p(t, x, 

In [98]:
prn2([f.subs(TD).doit().simplify() for f in f4])

0

tau^2 =>


0

h^2 =>


0

In [99]:
f1 = NF(F1, [u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [F2, F3, f4], head=True)
f1 = NF(f1, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [f1, F2, F3, f4])
prn2(f1)

Derivative(u(t, x, y), x) + Derivative(v(t, x, y), y)

tau^2 =>


0

h^2 =>


-Re*u(t, x, y)*Derivative(v(t, x, y), x, y)/6 - Re*v(t, x, y)*Derivative(v(t, x, y), (y, 2))/6 - Re*Derivative(p(t, x, y), (y, 2))/6 - Re*Derivative(u(t, x, y), y)*Derivative(v(t, x, y), x)/6 - Re*Derivative(v(t, x, y), y)**2/6 - Re*Derivative(v(t, x, y), t, y)/6 + Derivative(v(t, x, y), (y, 3))/3

In [100]:
prn2([f.subs(TD).doit().simplify() for f in f1])

0

tau^2 =>


0

h^2 =>


0

In [101]:
f2 = NF(F2, [u.diff(x), v.diff(x, 2), p.diff(x, 2)], [f1, F3, f4], head=True)
f2 = NF(f2, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [f1, f2, F3, f4])
prn2(f2)

-u(t, x, y)*Derivative(v(t, x, y), y) + v(t, x, y)*Derivative(u(t, x, y), y) + Derivative(p(t, x, y), x) + Derivative(u(t, x, y), t) - Derivative(u(t, x, y), (y, 2))/Re + Derivative(v(t, x, y), x, y)/Re

tau^2 =>


-Derivative(u(t, x, y), (t, 3))/12

h^2 =>


-Re**2*u(t, x, y)*v(t, x, y)**2*Derivative(v(t, x, y), y)/12 + Re**2*v(t, x, y)**3*Derivative(u(t, x, y), y)/12 + Re**2*v(t, x, y)**2*Derivative(p(t, x, y), x)/12 + Re**2*v(t, x, y)**2*Derivative(u(t, x, y), t)/12 - Re*u(t, x, y)**2*Derivative(v(t, x, y), x, y)/12 - Re*u(t, x, y)*v(t, x, y)*Derivative(v(t, x, y), (y, 2))/6 - Re*u(t, x, y)*Derivative(p(t, x, y), (y, 2))/12 - Re*u(t, x, y)*Derivative(u(t, x, y), y)*Derivative(v(t, x, y), x)/2 - Re*u(t, x, y)*Derivative(v(t, x, y), y)**2/2 + Re*v(t, x, y)**2*Derivative(v(t, x, y), x, y)/12 + Re*v(t, x, y)*Derivative(p(t, x, y), x, y)/12 + 5*Re*Derivative(p(t, x, y), x)*Derivative(v(t, x, y), y)/12 - 5*Re*Derivative(p(t, x, y), y)*Derivative(u(t, x, y), y)/12 + Re*Derivative(u(t, x, y), t)*Derivative(v(t, x, y), y)/2 - Re*Derivative(u(t, x, y), (t, 2))/12 - Re*Derivative(u(t, x, y), y)*Derivative(v(t, x, y), t)/2 - Re*Derivative(p(t, x, y), t, x)/12 + Derivative(u(t, x, y), (t, 5))/96 + Derivative(u(t, x, y), y)*Derivative(v(t, x, y), (y, 

In [102]:
prn2([f.subs(TD).doit().simplify() for f in f2])

0

tau^2 =>


-2*exp(-2*t/Re)*sin(y)*cos(x)/(3*Re**3)

h^2 =>


(Re**4 + 2)*exp(-2*t/Re)*sin(y)*cos(x)/(6*Re**5)

In [103]:
f3 = NF(F3, [u.diff(x), u.diff(y, 2), p.diff(x, 2)], [f1, f2, f4], head=True)
f3 = NF(f3, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [f1, f2, f3, f4])
prn(f3)

p_{y} + u*v_{x} + v*v_{y} + v_{t} - v_{xx}/Re - v_{yy}/Re

tau =>


-v_{ttt}/12

h^2 =>


Re**2*p_{y}*u**2/12 + Re**2*u**3*v_{x}/12 + Re**2*u**2*v*v_{y}/12 + Re**2*u**2*v_{t}/12 - Re*p_{ty}/12 + Re*p_{xy}*u/12 + Re*p_{x}*v_{x}/4 - Re*p_{yy}*v/12 - 5*Re*p_{y}*v_{y}/12 - Re*u**2*v_{yy}/12 - 2*Re*u*v_{x}*v_{y}/3 + Re*u_{t}*v_{x}/6 + Re*u_{y}*v*v_{x}/6 - Re*v**2*v_{yy}/12 - Re*v*v_{ty}/6 - Re*v*v_{y}**2/2 - Re*v_{tt}/12 - Re*v_{t}*v_{y}/2 + p_{yyy}/3 + u_{y}*v_{xy}/3 + v*v_{yyy}/3 + v_{ttttt}/96 + v_{tyy}/6 - v_{xy}*v_{x}/3 + 2*v_{yy}*v_{y} - v_{yyyy}/(6*Re)

In [104]:
prn2([f.subs(TD).doit().simplify() for f in f3])

0

tau^2 =>


2*exp(-2*t/Re)*sin(x)*cos(y)/(3*Re**3)

h^2 =>


-(Re**4 + 2)*exp(-2*t/Re)*sin(x)*cos(y)/(6*Re**5)

In [105]:
spoly = [a.diff(x) + b.diff(y) for a, b in zip(f2, f3)]
spoly = NF(spoly, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [f1, f2, f3, f4], head=True)
prn2(spoly)

0

tau^2 =>


0

h^2 =>


0